# 04: Merge first VRS "planned" data with stations in Bonn

Data that will be aggreagated, cleaned and merged in this notebook:
- Bonn Stations (Notebook 1)
- Agency
- Routes
- Stops
- Stop Times


### Note: this notebooks was executed on local machine

In [37]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
from shapely.geometry import Point
import hashlib

In [38]:
# Set month
data_month_single = '7'
data_month_double = '07'
data_month_int = 7

In [39]:
# load agency data
agency = pd.DataFrame(pd.read_csv("../Data/2023"+ data_month_double+"01-000000-UTC_gtfs-scraper_agency-vrs.csv"))

In [40]:
#calendar_dates = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/Raw Data/"+ data_month_double+"/VRS Planned Data/2023"+ data_month_double+"01-000000-UTC_gtfs-scraper_calendar_dates-vrs.csv"))

In [41]:
#calendar = pd.DataFrame(pd.read_csv("/Volumes/T7/Master/Raw Data/"+ data_month_double+"/VRS Planned Data/2023"+ data_month_double+"01-000000-UTC_gtfs-scraper_calendar-vrs.csv"))


## Routes

In [42]:
# load routes data
routes = pd.DataFrame(pd.read_csv("../Data/2023"+ data_month_double+"01-000000-UTC_gtfs-scraper_routes-vrs.csv"))
routes

,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,100001,1,1,NaN,0,E0071C,FFFFFF
1,100002,1,E,NaN,0,000000,FFFFFF
2,100003,1,3,NaN,0,F29DC2,FFFFFF
3,100004,1,4,NaN,0,EA71A5,FFFFFF
4,100005,1,5,NaN,0,A69CC8,FFFFFF
...,...,...,...,...,...,...,...
607,19901298,199,SEV,NaN,3,000000,FFFFFF
608,19911310,199,A,NaN,3,000000,FFFFFF
609,19911789,199,B,NaN,3,000000,FFFFFF
610,19911790,139,RB48,NaN,3,FFC82E,FFFFFF


In [43]:
# if short name is missing, fill with long name
routes['route_short_name'] = routes['route_short_name'].fillna(routes['route_long_name'])

In [44]:
# drop columns
routes.drop(columns=['route_long_name', 'route_color', 'route_text_color'], inplace=True)

In [45]:
routes

,route_id,agency_id,route_short_name,route_type
0,100001,1,1,0
1,100002,1,E,0
2,100003,1,3,0
3,100004,1,4,0
4,100005,1,5,0
...,...,...,...,...
607,19901298,199,SEV,3
608,19911310,199,A,3
609,19911789,199,B,3
610,19911790,139,RB48,3


In [46]:
# create column with route type name for better understanding
routes['route_type_name'] = routes['route_type'].apply(lambda x: 'Tram' if x == 0 else 'Rail' if x == 2 else 'Bus')

## Merge Agency with Routes

In [47]:
routes = routes.merge(agency[['agency_id', 'agency_name']], on='agency_id', how='left')
routes

,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name
0,100001,1,1,0,Tram,KVB Kölner Verkehrs-Betriebe AG
1,100002,1,E,0,Tram,KVB Kölner Verkehrs-Betriebe AG
2,100003,1,3,0,Tram,KVB Kölner Verkehrs-Betriebe AG
3,100004,1,4,0,Tram,KVB Kölner Verkehrs-Betriebe AG
4,100005,1,5,0,Tram,KVB Kölner Verkehrs-Betriebe AG
...,...,...,...,...,...,...
607,19901298,199,SEV,3,Bus,SEV Schienenersatzverkehr Deutsche Bahn AG
608,19911310,199,A,3,Bus,SEV Schienenersatzverkehr Deutsche Bahn AG
609,19911789,199,B,3,Bus,SEV Schienenersatzverkehr Deutsche Bahn AG
610,19911790,139,RB48,3,Bus,NX National Express Rail GmbH


In [48]:
# export to csv
routes.to_csv('../Data/04_routes_merged.csv', index=False)

Now, agency dataframe can be ignored

## Stops

In [49]:
# load stops data
stops = pd.DataFrame(pd.read_csv("../Data/2023"+data_month_double+"01-000000-UTC_gtfs-scraper_stops-vrs.csv"))
stops

,stop_id,stop_name,stop_lat,stop_lon,location_type
0,1,Köln Heumarkt,50.935705,6.959995,0
1,2,Köln Neumarkt,50.935770,6.947677,0
2,3,Köln Poststr.,50.931238,6.950060,0
3,4,Köln Mauritiuskirche,50.934173,6.945069,0
4,6,Köln Rathaus,50.938467,6.959697,0
...,...,...,...,...,...
6956,64728,Dormagen Dantestr.,51.112985,6.757370,0
6957,66006,Remscheid Kräwinklerbrücke,51.184089,7.305355,0
6958,74228,Wuppertal Kurvenstraße,51.248001,7.287557,0
6959,77620,Wuppertal In Der Hardt,51.220275,7.287913,0


In [50]:
#drop location_type
stops.drop(columns=['location_type'], inplace=True)

Now we need to narrow stops down with the stops that are only in Bonn

In [51]:

#load bonn stops
bonn_stops = gpd.read_parquet('../Data/01_bonn_stop_buffer.parquet')
bonn_stops = gpd.GeoDataFrame(bonn_stops)

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2915/120213552.py:2: UserWarning: Multiple non-primary geometry columns read from Parquet/Feather file. The first column read was promoted to the primary geometry.
  bonn_stops = gpd.read_parquet('/Volumes/T7/Master/Processed Data/01_bonn_stop_buffer.parquet')


In [52]:
# only keep stops in Bonn
stops = stops[stops['stop_name'].str.split().str[0] == 'Bonn']

In [53]:
stops

,stop_id,stop_name,stop_lat,stop_lon
39,43,Bonn Propsthof Nord,50.739096,7.073507
156,161,Bonn Bad Godesberg Bf,50.683920,7.159553
357,371,Bonn Bad Godesberg Stadthalle,50.680336,7.158347
653,683,Bonn Museum Koenig,50.721748,7.114519
654,684,Bonn Bundesrechnungshof/Auswärtiges Amt,50.725560,7.111579
...,...,...,...,...
6655,9716,Bonn Gerhart-Hauptmann-Str.,50.735969,7.060630
6688,9777,Bonn Kleine Str.,50.741163,7.047926
6689,9778,Bonn Magdalenenstr.,50.727103,7.072652
6690,9779,Bonn Am Dickobskreuz,50.737108,7.072725


In [54]:
# rename columns to prepare for merge
stops.rename(columns={'stop_lon': 'long'}, inplace=True)
stops.rename(columns={'stop_lat': 'lat'}, inplace=True)

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2915/4187973222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stops.rename(columns={'stop_lon': 'long'}, inplace=True)
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2915/4187973222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stops.rename(columns={'stop_lat': 'lat'}, inplace=True)


In [55]:
stops

,stop_id,stop_name,lat,long
39,43,Bonn Propsthof Nord,50.739096,7.073507
156,161,Bonn Bad Godesberg Bf,50.683920,7.159553
357,371,Bonn Bad Godesberg Stadthalle,50.680336,7.158347
653,683,Bonn Museum Koenig,50.721748,7.114519
654,684,Bonn Bundesrechnungshof/Auswärtiges Amt,50.725560,7.111579
...,...,...,...,...
6655,9716,Bonn Gerhart-Hauptmann-Str.,50.735969,7.060630
6688,9777,Bonn Kleine Str.,50.741163,7.047926
6689,9778,Bonn Magdalenenstr.,50.727103,7.072652
6690,9779,Bonn Am Dickobskreuz,50.737108,7.072725


In [56]:
# Create a new column "geometry" with Point objects
stops['geometry'] = stops.apply(lambda row: Point(row['long'], row['lat']), axis=1)

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2915/1411703467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stops['geometry'] = stops.apply(lambda row: Point(row['long'], row['lat']), axis=1)


In [57]:
stops

,stop_id,stop_name,lat,long,geometry
39,43,Bonn Propsthof Nord,50.739096,7.073507,POINT (7.073507 50.739096)
156,161,Bonn Bad Godesberg Bf,50.683920,7.159553,POINT (7.159553 50.68392)
357,371,Bonn Bad Godesberg Stadthalle,50.680336,7.158347,POINT (7.158347 50.680336)
653,683,Bonn Museum Koenig,50.721748,7.114519,POINT (7.114519 50.721748)
654,684,Bonn Bundesrechnungshof/Auswärtiges Amt,50.725560,7.111579,POINT (7.111579 50.72556)
...,...,...,...,...,...
6655,9716,Bonn Gerhart-Hauptmann-Str.,50.735969,7.060630,POINT (7.06063 50.735969)
6688,9777,Bonn Kleine Str.,50.741163,7.047926,POINT (7.047926 50.741163)
6689,9778,Bonn Magdalenenstr.,50.727103,7.072652,POINT (7.072652 50.727103)
6690,9779,Bonn Am Dickobskreuz,50.737108,7.072725,POINT (7.072725 50.737108)


In [58]:
# iterate over all stops and check if they are in buffer_zone
for index, stop_row in bonn_stops.iterrows():

    for index, stop in stops.iterrows():

        point = stop['geometry']

        if stop_row['buffer_zone'].contains(point):
           bonn_stops['stop_id'] = stops['stop_id']
           bonn_stops['stop_name'] = stops['stop_name']


In [59]:
bonn_stops

,station_point,buffer_zone,lat,long,stop_id,stop_name
39,POINT (7.07351 50.73910),"POLYGON ((7.07626 50.73943, 7.07630 50.73926, ...",50.739096,7.073507,43.0,Bonn Propsthof Nord
156,POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ...",50.683920,7.159553,161.0,Bonn Bad Godesberg Bf
357,POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ...",50.680336,7.158347,371.0,Bonn Bad Godesberg Stadthalle
600,POINT (7.00271 50.75686),"POLYGON ((7.00547 50.75720, 7.00550 50.75703, ...",50.756861,7.002714,NaN,NaN
643,POINT (7.00758 50.75425),"POLYGON ((7.01033 50.75458, 7.01037 50.75441, ...",50.754248,7.007576,NaN,NaN
...,...,...,...,...,...,...
6689,POINT (7.07265 50.72710),"POLYGON ((7.07540 50.72744, 7.07544 50.72727, ...",50.727103,7.072652,9778.0,Bonn Magdalenenstr.
6690,POINT (7.07273 50.73711),"POLYGON ((7.07548 50.73744, 7.07551 50.73727, ...",50.737108,7.072725,9779.0,Bonn Am Dickobskreuz
6691,POINT (7.05240 50.75620),"POLYGON ((7.05515 50.75653, 7.05519 50.75636, ...",50.756198,7.052400,9780.0,Bonn Georg-Elser-Weg
6716,POINT (7.01639 50.71882),"POLYGON ((7.01914 50.71915, 7.01918 50.71898, ...",50.718815,7.016388,NaN,NaN


In [60]:
# drop rows with missing values - those are not in Bonn
bonn_stops.dropna(subset=['stop_id', 'stop_name'], inplace=True)

In [61]:
bonn_stops

,station_point,buffer_zone,lat,long,stop_id,stop_name
39,POINT (7.07351 50.73910),"POLYGON ((7.07626 50.73943, 7.07630 50.73926, ...",50.739096,7.073507,43.0,Bonn Propsthof Nord
156,POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ...",50.683920,7.159553,161.0,Bonn Bad Godesberg Bf
357,POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ...",50.680336,7.158347,371.0,Bonn Bad Godesberg Stadthalle
653,POINT (7.11452 50.72175),"POLYGON ((7.11727 50.72208, 7.11731 50.72191, ...",50.721748,7.114519,683.0,Bonn Museum Koenig
654,POINT (7.11158 50.72556),"POLYGON ((7.11433 50.72589, 7.11437 50.72572, ...",50.725560,7.111579,684.0,Bonn Bundesrechnungshof/Auswärtiges Amt
...,...,...,...,...,...,...
6655,POINT (7.06063 50.73597),"POLYGON ((7.06338 50.73630, 7.06342 50.73613, ...",50.735969,7.060630,9716.0,Bonn Gerhart-Hauptmann-Str.
6688,POINT (7.04793 50.74116),"POLYGON ((7.05068 50.74150, 7.05072 50.74133, ...",50.741163,7.047926,9777.0,Bonn Kleine Str.
6689,POINT (7.07265 50.72710),"POLYGON ((7.07540 50.72744, 7.07544 50.72727, ...",50.727103,7.072652,9778.0,Bonn Magdalenenstr.
6690,POINT (7.07273 50.73711),"POLYGON ((7.07548 50.73744, 7.07551 50.73727, ...",50.737108,7.072725,9779.0,Bonn Am Dickobskreuz


In [62]:
# convert to int
bonn_stops['stop_id'] = bonn_stops['stop_id'].astype(int)

In [63]:
bonn_stops

,station_point,buffer_zone,lat,long,stop_id,stop_name
39,POINT (7.07351 50.73910),"POLYGON ((7.07626 50.73943, 7.07630 50.73926, ...",50.739096,7.073507,43,Bonn Propsthof Nord
156,POINT (7.15955 50.68392),"POLYGON ((7.16230 50.68425, 7.16234 50.68408, ...",50.683920,7.159553,161,Bonn Bad Godesberg Bf
357,POINT (7.15835 50.68034),"POLYGON ((7.16110 50.68067, 7.16113 50.68050, ...",50.680336,7.158347,371,Bonn Bad Godesberg Stadthalle
653,POINT (7.11452 50.72175),"POLYGON ((7.11727 50.72208, 7.11731 50.72191, ...",50.721748,7.114519,683,Bonn Museum Koenig
654,POINT (7.11158 50.72556),"POLYGON ((7.11433 50.72589, 7.11437 50.72572, ...",50.725560,7.111579,684,Bonn Bundesrechnungshof/Auswärtiges Amt
...,...,...,...,...,...,...
6655,POINT (7.06063 50.73597),"POLYGON ((7.06338 50.73630, 7.06342 50.73613, ...",50.735969,7.060630,9716,Bonn Gerhart-Hauptmann-Str.
6688,POINT (7.04793 50.74116),"POLYGON ((7.05068 50.74150, 7.05072 50.74133, ...",50.741163,7.047926,9777,Bonn Kleine Str.
6689,POINT (7.07265 50.72710),"POLYGON ((7.07540 50.72744, 7.07544 50.72727, ...",50.727103,7.072652,9778,Bonn Magdalenenstr.
6690,POINT (7.07273 50.73711),"POLYGON ((7.07548 50.73744, 7.07551 50.73727, ...",50.737108,7.072725,9779,Bonn Am Dickobskreuz


In [64]:
# export to parquet
bonn_stops.to_parquet('../Data/04_bonn_stations.parquet') 

## Cut "Stop Times" and megre with stop data from above

Trim Stop Times to stops only in Bonn

In [65]:
# load stop times data
stop_times =pd.DataFrame(pd.read_csv("../Data/2023"+ data_month_double+"01-000000-UTC_gtfs-scraper_stop_times-vrs.csv"))
stop_times

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2915/705314464.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times =pd.DataFrame(pd.read_csv("/Volumes/T7/Master/Raw Data/"+ data_month_double+"/VRS Planned data/2023"+ data_month_double+"01-000000-UTC_gtfs-scraper_stop_times-vrs.csv"))


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
0,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:15:00,12:15:00,8279,1,NaN,0,0
1,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:17:00,12:17:00,4145,2,NaN,0,0
2,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:19:00,12:19:00,3642,3,NaN,0,0
3,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:21:00,12:21:00,3911,4,NaN,0,0
4,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:22:00,12:22:00,3817,5,NaN,0,0
...,...,...,...,...,...,...,...,...
2444727,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:00:00,28:00:00,1956,31,NaN,0,0
2444728,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:02:00,28:02:00,3100,32,NaN,0,0
2444729,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:03:00,28:03:00,5258,33,NaN,0,0
2444730,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:04:00,28:04:00,3433,34,NaN,0,0


In [66]:
# drop columns
stop_times.drop(columns=['pickup_type', 'drop_off_type'], inplace=True)

In [67]:
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign
0,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:15:00,12:15:00,8279,1,NaN
1,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:17:00,12:17:00,4145,2,NaN
2,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:19:00,12:19:00,3642,3,NaN
3,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:21:00,12:21:00,3911,4,NaN
4,9-398-011-8279.2.21:121500-23-1_93B3E758-D5F5-...,12:22:00,12:22:00,3817,5,NaN
...,...,...,...,...,...,...
2444727,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:00:00,28:00:00,1956,31,NaN
2444728,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:02:00,28:02:00,3100,32,NaN
2444729,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:03:00,28:03:00,5258,33,NaN
2444730,7-288-016-3432.2.21:272100-35-1_335F78EC-5757-...,28:04:00,28:04:00,3433,34,NaN


In [68]:

# Merge the stop_times and merged_stops DataFrames based on the stop_id column
merged_data = pd.merge(stop_times, bonn_stops, on='stop_id', how='inner')

# The merged_data DataFrame will now contain only the rows where the stop_id matches in both DataFrames

In [69]:
merged_data

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,station_point,buffer_zone,lat,long,stop_name
0,6330006-633-006-1584.2.24:202100-11-1_A90CDB5F...,20:21:00,20:21:00,1584,1,NaN,POINT (7.15513 50.72238),"POLYGON ((7.15789 50.72271, 7.15792 50.72254, ...",50.722377,7.155135,Bonn Ramersdorf
1,6330006-633-006-1584.2.24:202100-11-1_A90CDB5F...,20:22:00,20:22:00,1583,2,NaN,POINT (7.15079 50.72595),"POLYGON ((7.15354 50.72629, 7.15358 50.72611, ...",50.725953,7.150790,Bonn Samansstr.
2,6330006-633-006-1584.2.24:202100-11-1_A90CDB5F...,20:24:00,20:24:00,1582,3,NaN,POINT (7.14873 50.73034),"POLYGON ((7.15148 50.73068, 7.15152 50.73050, ...",50.730342,7.148731,Bonn Küdinghoven Kirche
3,6330006-633-006-1584.2.24:202100-11-1_A90CDB5F...,20:25:00,20:25:00,1581,4,NaN,POINT (7.14563 50.73330),"POLYGON ((7.14838 50.73364, 7.14842 50.73346, ...",50.733303,7.145630,Bonn Forstweg
4,6330006-633-006-1584.2.24:202100-11-1_A90CDB5F...,20:26:00,20:26:00,1580,5,NaN,POINT (7.14250 50.73497),"POLYGON ((7.14525 50.73530, 7.14529 50.73513, ...",50.734970,7.142498,Bonn Broichstr.
...,...,...,...,...,...,...,...,...,...,...,...
354971,6350145-635-006-1584.2.23:200800-19-1_BB9DD49C...,20:20:00,20:20:00,1568,12,NaN,POINT (7.16736 50.74520),"POLYGON ((7.17011 50.74553, 7.17015 50.74536, ...",50.745197,7.167358,Bonn Am Rehsprung
354972,6350145-635-006-1584.2.23:200800-19-1_BB9DD49C...,20:21:00,20:21:00,1566,13,NaN,POINT (7.17369 50.74516),"POLYGON ((7.17644 50.74549, 7.17648 50.74532, ...",50.745160,7.173690,Bonn Paul-Langen-Str.
354973,6350145-635-006-1584.2.23:200800-19-1_BB9DD49C...,20:23:00,20:23:00,1562,14,NaN,POINT (7.17490 50.75007),"POLYGON ((7.17766 50.75041, 7.17769 50.75023, ...",50.750074,7.174904,Bonn Kohlkauler Platz
354974,6350145-635-006-1584.2.23:200800-19-1_BB9DD49C...,20:24:00,20:24:00,2313,15,NaN,POINT (7.16800 50.75066),"POLYGON ((7.17076 50.75099, 7.17079 50.75082, ...",50.750656,7.168004,Bonn Ennertstr.


In [70]:
# create a GeoDataFrame
merged_data = gpd.GeoDataFrame(merged_data)

In [71]:
merged_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 354976 entries, 0 to 354975
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   trip_id         354976 non-null  object  
 1   arrival_time    354976 non-null  object  
 2   departure_time  354976 non-null  object  
 3   stop_id         354976 non-null  int64   
 4   stop_sequence   354976 non-null  int64   
 5   stop_headsign   216 non-null     object  
 6   station_point   354976 non-null  geometry
 7   buffer_zone     354976 non-null  geometry
 8   lat             354976 non-null  float64 
 9   long            354976 non-null  float64 
 10  stop_name       354976 non-null  object  
dtypes: float64(2), geometry(2), int64(2), object(5)
memory usage: 29.8+ MB


In [72]:
# export the merged data to parquet
merged_data.to_parquet('../Data/04_merged_stop_times_buffer_zones.parquet') 